In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from typing import List, Tuple
import numpy as np
import rdkit
import pandas as pd
from pymatgen.core import Structure, Lattice, Molecule, Element

%load_ext autoreload
%autoreload 2

smiles = 'CC(C)C'

m = Chem.MolFromSmiles(smiles)
# necessary to add hydrogen for consistent conformer generation
m = Chem.AddHs(m)





In [13]:
import json
import gzip
from pymatgen.core.structure import Structure, Molecule

from utils import download_url

#raw_url= "https://ml.materialsproject.org/projects/matbench_mp_is_metal.json.gz"
#raw_dir= "../data/matbench/mp_is_metal"
#download_url(raw_url, raw_dir)


json_filename = "../data/matbench/mp_is_metal/raw/matbench_mp_is_metal.json.gz"


with gzip.open(json_filename, 'r') as fin:        # 4. gzip
    json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)

json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
data = json.loads(json_str) 



df = pd.DataFrame({"struct":[data_list[0] for data_list in data["data"]], "target":[data_list[1] for data_list in data["data"]]})

df.set_index("struct")

struct= Structure.from_dict(data["data"][0][0])

In [4]:
structure.formula

'K2 Na1'

In [6]:
from utils import from_structure_to_molecule


mol = from_structure_to_molecule(struct=structure, add_hydrogen=False)

In [26]:
a = np.array([0,0,1,1,2,2])
b = np.array([1,2,0,2,0,1])

structure.distance_matrix[a,b]

In [12]:
from utils import  Distance, from_structure_to_molecule, from_molecule_to_graph
import torch

atom_number_to_radius = torch.load("../important_data/atom_number_to_radius.pt")
    
distance = Distance(atom_number_to_radius=atom_number_to_radius)

from datasets_classes import BaceDataset

dataset = BaceDataset(root="../data/bace/")
graph = dataset[0]


#mol = from_structure_to_molecule(struct=structure,add_hydrogen=False)
#graph = from_molecule_to_graph(mol=mol, y=None, )

## given the pos, we need to see what dist is computed, maybe just give struct.frac_coords, need to check the format with real smiles molecule

(row, col), pos, pseudo = graph.edge_index, graph.pos, graph.edge_attr
dist = torch.norm(pos[col] - pos[row], p=2, dim=-1).view(-1, 1)

In [1]:
from datasets_classes import MatBenchMpIsMetal

dataset = MatBenchMpIsMetal(root="../data/matbench/mp_is_metal/")

In [33]:
Descriptors._rdMolDescriptors

<module 'rdkit.Chem.rdMolDescriptors' from '/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/rdkit/Chem/rdMolDescriptors.so'>

In [63]:
## obtained from Descriptors.__dict__.keys()
## filtered to keep only useful

from rdkit.Chem import Descriptors, Descriptors3D

##BCUT and PartialCharge features are problematic for crystallographic derived molecules



funcs_descriptor_2d = ['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 
           'qed', '_isCallable', '_descList', '_setupDescriptors', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 
           '_ChargeDescriptors', '_FingerprintDensity', 'FpDensityMorgan1',
           'FpDensityMorgan2', 'FpDensityMorgan3', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n',
           'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 
           'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 
           'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 
           'SlogP_VSA12', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA9', 'TPSA', 
           'EState_VSA1', 'EState_VSA10', 'EState_VSA11', 'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 
           'EState_VSA8', 'EState_VSA9', 'VSA_EState1', 'VSA_EState10', 'VSA_EState2', 'VSA_EState3', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 
           'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'FractionCSP3', 'HeavyAtomCount', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
           'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 
           'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
           'NumSaturatedRings', 'RingCount', 'MolLogP', 'MolMR', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO', 
           'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine', 
           'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O', 'fr_Ndealkylation1', 'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde',
           'fr_alkyl_carbamate', 'fr_alkyl_halide', 'fr_allylic_oxid', 'fr_amide', 'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 
           'fr_azide', 'fr_azo', 'fr_barbitur', 'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic', 'fr_diazo', 'fr_dihydropyridine', 
           'fr_epoxide', 'fr_ester', 'fr_ether', 'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone', 'fr_imidazole', 'fr_imide', 
           'fr_isocyan', 'fr_isothiocyan', 'fr_ketone', 'fr_ketone_Topliss', 'fr_lactam', 'fr_lactone', 'fr_methoxy', 'fr_morpholine', 'fr_nitrile',
           'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 'fr_phenol', 
           'fr_phenol_noOrthoHbond', 'fr_phos_acid', 'fr_phos_ester', 'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd', 'fr_pyridine', 
           'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 'fr_thiophene',
           'fr_unbrch_alkane', 'fr_urea', 'AUTOCORR2D_1', 'AUTOCORR2D_2', 'AUTOCORR2D_3', 'AUTOCORR2D_4', 'AUTOCORR2D_5', 'AUTOCORR2D_6', 
           'AUTOCORR2D_7', 'AUTOCORR2D_8', 'AUTOCORR2D_9', 'AUTOCORR2D_10', 'AUTOCORR2D_11', 'AUTOCORR2D_12', 'AUTOCORR2D_13', 'AUTOCORR2D_14', 
           'AUTOCORR2D_15', 'AUTOCORR2D_16', 'AUTOCORR2D_17', 'AUTOCORR2D_18', 'AUTOCORR2D_19', 'AUTOCORR2D_20', 'AUTOCORR2D_21', 'AUTOCORR2D_22', 
           'AUTOCORR2D_23', 'AUTOCORR2D_24', 'AUTOCORR2D_25', 'AUTOCORR2D_26', 'AUTOCORR2D_27', 'AUTOCORR2D_28', 'AUTOCORR2D_29', 'AUTOCORR2D_30', 
           'AUTOCORR2D_31', 'AUTOCORR2D_32', 'AUTOCORR2D_33', 'AUTOCORR2D_34', 'AUTOCORR2D_35', 'AUTOCORR2D_36', 'AUTOCORR2D_37', 'AUTOCORR2D_38', 
           'AUTOCORR2D_39', 'AUTOCORR2D_40', 'AUTOCORR2D_41', 'AUTOCORR2D_42', 'AUTOCORR2D_43', 'AUTOCORR2D_44', 'AUTOCORR2D_45', 'AUTOCORR2D_46', 
           'AUTOCORR2D_47', 'AUTOCORR2D_48', 'AUTOCORR2D_49', 'AUTOCORR2D_50', 'AUTOCORR2D_51', 'AUTOCORR2D_52', 'AUTOCORR2D_53', 'AUTOCORR2D_54', 
           'AUTOCORR2D_55', 'AUTOCORR2D_56', 'AUTOCORR2D_57', 'AUTOCORR2D_58', 'AUTOCORR2D_59', 'AUTOCORR2D_60', 'AUTOCORR2D_61', 'AUTOCORR2D_62', 
           'AUTOCORR2D_63', 'AUTOCORR2D_64', 'AUTOCORR2D_65', 'AUTOCORR2D_66', 'AUTOCORR2D_67', 'AUTOCORR2D_68', 'AUTOCORR2D_69', 'AUTOCORR2D_70', 
           'AUTOCORR2D_71', 'AUTOCORR2D_72', 'AUTOCORR2D_73', 'AUTOCORR2D_74', 'AUTOCORR2D_75', 'AUTOCORR2D_76', 'AUTOCORR2D_77', 'AUTOCORR2D_78', 
           'AUTOCORR2D_79', 'AUTOCORR2D_80', 'AUTOCORR2D_81', 'AUTOCORR2D_82', 'AUTOCORR2D_83', 'AUTOCORR2D_84', 'AUTOCORR2D_85', 'AUTOCORR2D_86', 
           'AUTOCORR2D_87', 'AUTOCORR2D_88', 'AUTOCORR2D_89', 'AUTOCORR2D_90', 'AUTOCORR2D_91', 'AUTOCORR2D_92', 'AUTOCORR2D_93', 'AUTOCORR2D_94', 
           'AUTOCORR2D_95', 'AUTOCORR2D_96', 'AUTOCORR2D_97', 'AUTOCORR2D_98', 'AUTOCORR2D_99', 'AUTOCORR2D_100', 'AUTOCORR2D_101', 'AUTOCORR2D_102', 
           'AUTOCORR2D_103', 'AUTOCORR2D_104', 'AUTOCORR2D_105', 'AUTOCORR2D_106', 'AUTOCORR2D_107', 'AUTOCORR2D_108', 'AUTOCORR2D_109', 'AUTOCORR2D_110', 
           'AUTOCORR2D_111', 'AUTOCORR2D_112', 'AUTOCORR2D_113', 'AUTOCORR2D_114', 'AUTOCORR2D_115', 'AUTOCORR2D_116', 'AUTOCORR2D_117', 'AUTOCORR2D_118', 
           'AUTOCORR2D_119', 'AUTOCORR2D_120', 'AUTOCORR2D_121', 'AUTOCORR2D_122', 'AUTOCORR2D_123', 'AUTOCORR2D_124', 'AUTOCORR2D_125', 'AUTOCORR2D_126', 
           'AUTOCORR2D_127', 'AUTOCORR2D_128', 'AUTOCORR2D_129', 'AUTOCORR2D_130', 'AUTOCORR2D_131', 'AUTOCORR2D_132', 'AUTOCORR2D_133', 'AUTOCORR2D_134', 
           'AUTOCORR2D_135', 'AUTOCORR2D_136', 'AUTOCORR2D_137', 'AUTOCORR2D_138', 'AUTOCORR2D_139', 'AUTOCORR2D_140', 'AUTOCORR2D_141', 'AUTOCORR2D_142', 
           'AUTOCORR2D_143', 'AUTOCORR2D_144', 'AUTOCORR2D_145', 'AUTOCORR2D_146', 'AUTOCORR2D_147', 'AUTOCORR2D_148', 'AUTOCORR2D_149', 'AUTOCORR2D_150', 
           'AUTOCORR2D_151', 'AUTOCORR2D_152', 'AUTOCORR2D_153', 'AUTOCORR2D_154', 'AUTOCORR2D_155', 'AUTOCORR2D_156', 'AUTOCORR2D_157', 'AUTOCORR2D_158', 
           'AUTOCORR2D_159', 'AUTOCORR2D_160', 'AUTOCORR2D_161', 'AUTOCORR2D_162', 'AUTOCORR2D_163', 'AUTOCORR2D_164', 'AUTOCORR2D_165', 'AUTOCORR2D_166', 
           'AUTOCORR2D_167', 'AUTOCORR2D_168', 'AUTOCORR2D_169', 'AUTOCORR2D_170', 'AUTOCORR2D_171', 'AUTOCORR2D_172', 'AUTOCORR2D_173', 'AUTOCORR2D_174', 
           'AUTOCORR2D_175', 'AUTOCORR2D_176', 'AUTOCORR2D_177', 'AUTOCORR2D_178', 'AUTOCORR2D_179', 'AUTOCORR2D_180', 'AUTOCORR2D_181', 'AUTOCORR2D_182', 
           'AUTOCORR2D_183', 'AUTOCORR2D_184', 'AUTOCORR2D_185', 'AUTOCORR2D_186', 'AUTOCORR2D_187', 'AUTOCORR2D_188', 'AUTOCORR2D_189', 'AUTOCORR2D_190', 
           'AUTOCORR2D_191', 'AUTOCORR2D_192']

funcs_descriptor_2d = [x for x in funcs_descriptor_2d if '_' not in x or str.index(x,'_') != 0]

funcs_descriptor_3d = ['PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2', 'RadiusOfGyration', 'InertialShapeFactor', 'Eccentricity', 'Asphericity', 'SpherocityIndex']

In [68]:
len(x)

400

In [67]:
from utils import from_structure_to_molecule

from pymatgen.core.lattice import Lattice

structure = Structure(
    Lattice.cubic(4.2),
    ["Na", "K", "K"],
    [[0, 0, 0], [0.5, 0.5, 0.5], [0,0,0.5]],
    site_properties={"magmom": [-2, 2,2]},
)

structure = data["data"][0][0]

m = from_structure_to_molecule(struct=struct)




x=[]
for func_name in funcs_descriptor_2d:
    func= Descriptors.__dict__[func_name]
    res = func(m)
    if np.isnan(res):
        print(func_name)
    x.append(res)

for func_name in funcs_descriptor_3d:
    func = Descriptors3D.__dict__[func_name]
    x.append(func(m))


x

[14.601510022385533,
 -5.036958087671614,
 14.601510022385533,
 0.7026500544292307,
 0.5929347334781221,
 431.57999999999964,
 398.31600000000026,
 431.2572772960009,
 168,
 0,
 0.22288521095429528,
 -0.3834506314098386,
 0.3834506314098386,
 0.22288521095429528,
 1.53125,
 2.4375,
 3.25,
 2.2175513405828985e-15,
 -0.24714542964875752,
 -1.3233391105315108e-16,
 -2.7875976524389667,
 3.164688084617106e-33,
 -2.8568422717700646,
 3.863045167333953e-16,
 -1.1154727414736292,
 2.2519222220546284,
 2455.504783196726,
 51.30651840778636,
 48.6581373674276,
 15.658137367427601,
 28.60650713076904,
 24.822047210945563,
 8.48040642444569,
 6.059890118488227,
 6.059890118488227,
 3.7667207273600374,
 3.7667207273600374,
 2.3945307816806483,
 2.3945307816806483,
 -2.9199999999999995,
 4040364822576.555,
 5.4136917090175825,
 8.907698428298927,
 4.9370301265713135,
 236.46630066989687,
 15.766393282814699,
 5.817862777835028,
 4.235526234984602,
 5.907179729351506,
 0.0,
 0.0,
 4.794537184071822,

In [66]:
from datasets_classes import BaceDataset

dataset = BaceDataset(root="../data/bace/")

dataset[0]

Data(x=[32, 5], edge_index=[2, 992], edge_attr=[992, 5], y=[1, 2], pos=[65, 3], z=[32], name='O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C', idx=[1])

## Using Element featurization 

In [63]:
from utils import from_smiles_to_molecule_and_coordinates

smiles = 'O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C'

m,pos = from_smiles_to_molecule_and_coordinates(smile=smiles, add_hydrogen=False, seed=12)

from pymatgen.io.babel import BabelMolAdaptor

mol_file = Chem.MolToMolBlock(m)

pymatgen_mol = BabelMolAdaptor.from_string(string_data=mol_file, file_format="mol").pymatgen_mol

print([atom.GetAtomicNum() for atom in m.GetAtoms()])

## need to check atom ordering
elem = pymatgen_mol.species[0]
elem1 = pymatgen_mol.species[1]

#print(pymatgen_mol.species[1].__dict__)

[8, 6, 6, 6, 7, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [71]:

from utils import from_rdkit_mol_to_pymatgen_mol
import torch

def node_features(mol:Chem.rdchem.Mol):
    
    features = ["atomic_radius","atomic_mass","average_ionic_radius", "average_cationic_radius", "average_anionic_radius", "max_oxidation_state",
            "min_oxidation_state", "row","group", "is_noble_gas", "is_post_transition_metal", "is_rare_earth_metal", "is_metal", "is_metalloid",
            "is_alkali", "is_alkaline", "is_halogen","is_chalcogen", "is_lanthanoid","is_actinoid", "is_quadrupolar"] 
    
    features_dict = {feature:[] for feature in features}
    
    pymatgen_mol = from_rdkit_mol_to_pymatgen_mol(mol=mol)
    atomic_number = []
    for elem in pymatgen_mol.species:
        atomic_number.append(elem.Z)
        for feature in features:
            features_dict[feature].append(getattr(elem,feature))
            
    all_features = [feature_list for feature_list in features_dict.values()]
    x= [atomic_number] + all_features
    print(x)
    x = torch.tensor(x,dtype=torch.float).t().contiguous()
    z = torch.tensor(atomic_number, dtype=torch.long)
    
    return x,z

In [72]:
node_features(m)

[[8, 6, 6, 6, 7, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0.6, 0.7, 0.7, 0.7, 0.65, 0.7, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.65, 0.7, 0.65, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7], [15.9994, 12.0107, 12.0107, 12.0107, 14.0067, 12.0107, 15.9994, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 14.0067, 12.0107, 14.0067, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107, 12.0107], [1.26, 0.3, 0.3, 0.3, 0.63, 0.3, 1.26, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.63, 0.3, 0.63, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3], [0.0, 0.3, 0.3, 0.3, 0.28500000000000003, 0.3, 0.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.28500000000000003, 0.3, 0.28500000000000003, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3], [1.26, 0.0, 0.0, 0.0, 1.32, 0.0, 1.26, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

(tensor([[ 8.0000,  0.6000, 15.9994,  1.2600,  0.0000,  1.2600,  2.0000, -2.0000,
           2.0000, 16.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
         [ 6.0000,  0.7000, 12.0107,  0.3000,  0.3000,  0.0000,  4.0000, -4.0000,
           2.0000, 14.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
         [ 6.0000,  0.7000, 12.0107,  0.3000,  0.3000,  0.0000,  4.0000, -4.0000,
           2.0000, 14.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
         [ 6.0000,  0.7000, 12.0107,  0.3000,  0.3000,  0.0000,  4.0000, -4.0000,
           2.0000, 14.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
         [ 7.0000,  0.6500, 14.0067,  0.6300,  0.2850,  1.3200,  5.0000, -3.0000,
  

# Testing exsitence of node features for all elements

In [57]:
features = ["atomic_radius","atomic_mass","average_ionic_radius", "average_cationic_radius", "average_anionic_radius", "max_oxidation_state",
            "min_oxidation_state", "row","group", "is_noble_gas", "is_post_transition_metal", "is_rare_earth_metal", "is_metal", "is_metalloid",
            "is_alkali", "is_alkaline", "is_halogen","is_chalcogen", "is_lanthanoid","is_actinoid", "is_quadrupolar"] 

for i in range(1,118):
    elem = Element.from_Z(i)
    for feature in features:
        try:
            getattr(elem, feature)
        except:
            raise ValueError(f"{feature}")


In [58]:


for elem in pymatgen_mol.species[:10]:
    print([getattr(elem, key) for key in features])

[0.6, 15.9994, 1.26, 0.0, 1.26, 2, -2, 2, 16, False, False, False, False, False, False, False, False, True, False, False, True]
[0.7, 12.0107, 0.3, 0.3, 0.0, 4, -4, 2, 14, False, False, False, False, False, False, False, False, False, False, False, True]
[0.7, 12.0107, 0.3, 0.3, 0.0, 4, -4, 2, 14, False, False, False, False, False, False, False, False, False, False, False, True]
[0.7, 12.0107, 0.3, 0.3, 0.0, 4, -4, 2, 14, False, False, False, False, False, False, False, False, False, False, False, True]
[0.65, 14.0067, 0.63, 0.28500000000000003, 1.32, 5, -3, 2, 15, False, False, False, False, False, False, False, False, False, False, False, True]
[0.7, 12.0107, 0.3, 0.3, 0.0, 4, -4, 2, 14, False, False, False, False, False, False, False, False, False, False, False, True]
[0.6, 15.9994, 1.26, 0.0, 1.26, 2, -2, 2, 16, False, False, False, False, False, False, False, False, True, False, False, True]
[0.7, 12.0107, 0.3, 0.3, 0.0, 4, -4, 2, 14, False, False, False, False, False, False, Fals

In [56]:
elem.atomic_radius, elem.atomic_mass, elem.data

(0.6,
 15.9994,
 {'Atomic mass': 15.9994,
  'Atomic no': 8,
  'Atomic orbitals': {'1s': -18.758245, '2p': -0.338381, '2s': -0.871362},
  'Atomic radius': 0.6,
  'Atomic radius calculated': 0.48,
  'Boiling point': '90.2 K',
  'Brinell hardness': 'no data MN m<sup>-2</sup>',
  'Bulk modulus': 'no data GPa',
  'Coefficient of linear thermal expansion': 'no data x10<sup>-6</sup>K<sup>-1</sup>',
  'Common oxidation states': [-2],
  'Critical temperature': '154.6 K',
  'Density of solid': 'no data kg m<sup>-3</sup>',
  'Electrical resistivity': 'no data 10<sup>-8</sup> &Omega; m',
  'Electronic structure': '[He].2s<sup>2</sup>.2p<sup>4</sup>',
  'ICSD oxidation states': [-2],
  'Ionic radii': {'-2': 1.26},
  'Liquid range': '35.4 K',
  'Melting point': '54.8 K',
  'Mendeleev no': 101,
  'Mineral hardness': 'no data',
  'Molar volume': '17.36 cm<sup>3</sup>',
  'Name': 'Oxygen',
  'Oxidation states': [-2, -1, 1, 2],
  'Poissons ratio': 'no data',
  'Reflectivity': 'no data %',
  'Refractive 

In [34]:
Element.print_periodic_table()

H                                                                   He 
Li  Be                                          B   C   N   O   F   Ne 
Na  Mg                                          Al  Si  P   S   Cl  Ar 
K   Ca  Sc  Ti  V   Cr  Mn  Fe  Co  Ni  Cu  Zn  Ga  Ge  As  Se  Br  Kr 
Rb  Sr  Y   Zr  Nb  Mo  Tc  Ru  Rh  Pd  Ag  Cd  In  Sn  Sb  Te  I   Xe 
Cs  Ba      Hf  Ta  W   Re  Os  Ir  Pt  Au  Hg  Tl  Pb  Bi  Po  At  Rn 
Fr  Ra      Rf  Db  Sg  Bh  Hs  Mt  Ds  Rg  Cn  Nh  Fl  Mc  Lv  Ts  Og 
        La  Ce  Pr  Nd  Pm  Sm  Eu  Gd  Tb  Dy  Ho  Er  Tm  Yb  Lu     
        Ac  Th  Pa  U   Np  Pu  Am  Cm  Bk  Cf  Es  Fm  Md  No  Lr     
